In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-09"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
24597,2024-11-09,Portugal Lpb,15:00,Galitos Barreiro,CD Póvoa,1.56,2.21,148.5,1.85,1.81,-4.5,1.94,1.73,WvcofkMa,0.641026,0.452489,0.540541,0.552486,0.093514,197.260,94.409012,0.478602,1.2,1.643168,1.369306,139.40,2.734,1.856645,0.679095,-15.0,265.298,202.480551,0.763219,0.0,0.000000,NaN,598.40,3.968,3.262571,0.822221,-45.0,85,64,1.64,9.35,127.926,430.418,0.243830,0.015456,0.080922,0.64,0.128,4.375000,0.671175,0.9,0.228825,-5.00,-1.000,-1.210000,0.297005,0.3,0.002995
24598,2024-11-09,Portugal Lpb,18:30,Esgueira,Benfica,10.50,1.03,157.5,1.85,1.81,16.5,1.83,1.83,E3QKwUyJ,0.095238,0.970874,0.540541,0.552486,0.066112,297.376,242.003725,0.813797,2.4,1.341641,0.559017,235.34,3.572,2.924734,0.818795,31.0,101.796,11.130132,0.109338,3.0,0.000000,0.000000,118.90,1.222,0.207292,0.169633,62.0,82,82,2.87,1.45,310.860,97.472,1.161544,0.015456,0.000000,8.98,1.796,5.289532,0.000000,0.0,0.000000,0.64,0.128,0.234375,0.902987,0.9,-0.002987
24599,2024-11-09,Portugal Lpb,17:30,CA Queluz,Vitória SC,1.61,2.12,162.5,1.83,1.83,-3.5,1.91,1.76,tAI61XTP,0.621118,0.471698,0.546448,0.546448,0.092816,285.062,205.916607,0.722357,1.8,1.643168,0.912871,648.00,3.422,2.570033,0.751032,-7.0,414.192,383.197860,0.925170,0.0,0.000000,NaN,105.86,5.884,5.169631,0.878591,-73.0,81,79,8.00,1.34,156.928,418.928,0.193364,0.000000,0.057802,1.99,0.398,1.532663,0.000000,0.0,0.000000,-1.70,-0.340,-3.294118,0.373244,0.4,0.026756
24600,2024-11-09,Portugal Proliga,12:00,CB Queluz,Imortal B,1.56,2.20,143.5,1.82,1.84,-3.5,1.84,1.82,tYWCXlle,0.641026,0.454545,0.549451,0.543478,0.095571,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,161.458,42.615682,0.263943,3.0,0.000000,0.000000,119.07,2.132,0.596297,0.279689,49.0,79,81,2.20,1.47,0.000,174.978,0.240717,0.007728,0.007728,0.00,0.000,inf,0.000000,0.0,0.000000,3.38,0.676,1.775148,0.411787,0.4,-0.011787
24601,2024-11-09,Portugal Lpb,15:00,FC Porto,Oliveirense,1.33,2.95,152.5,1.80,1.86,-10.5,2.03,1.67,pOY3sCxf,0.751880,0.338983,0.555556,0.537634,0.090863,174.320,107.689792,0.617771,1.2,1.643168,1.369306,190.96,2.354,1.459582,0.620043,-3.0,293.700,284.631321,0.969123,1.8,1.643168,0.912871,97.20,4.082,4.358047,1.067625,46.0,77,90,2.48,1.08,99.198,324.466,0.535286,0.023184,0.137599,-1.82,-0.364,-0.906593,0.876300,0.8,-0.076300,-1.98,-0.396,-4.924242,0.512537,0.6,0.087463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24844,2024-11-09,Eua Ncaa,21:00,UTSA Roadrunners,North Dakota,1.26,3.55,155.5,1.85,1.85,-8.5,1.85,1.85,O0OHp8wb,0.793651,0.281690,0.540541,0.540541,0.075341,360.748,320.666036,0.888892,1.8,1.643168,0.912871,116.51,5.004,4.491028,0.897488,17.0,223.602,219.604488,0.982122,0.6,1.341641,2.236068,616.00,3.592,4.153718,1.156380,-13.0,61,56,1.91,11.00,180.312,288.834,0.673295,0.000000,0.000000,0.43,0.086,3.023256,0.470380,0.3,-0.170380,3.20,0.640,3.984375,0.332214,0.4,0.067786
24845,2024-11-09,Eua Ncaa,21:00,Wichita State,Montana State,1.12,4.97,155.5,1.87,1.79,-13.5,2.03,1.67,K81wS5vJ,0.892857,0.201207,0.534759,0.558659,0.094064,188.486,60.402729,0.320463,1.8,1.643168,0.912871,177.45,2.522,0.975

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
24597,15:00,Portugal Lpb,Galitos Barreiro,CD Póvoa,1.56,Back Home
24609,15:00,Portugal Proliga,Lusitania,Barreirense,2.38,Back Home
24635,05:05,Japão B.League,Nagasaki,Akita,1.62,Back Home
24666,16:00,Eua Ncaa,Drexel,Colgate,1.83,Back Home
24755,11:00,República Tcheca Nbl,Brno,Srsni Pisek,2.08,Back Home
24761,14:00,Espanha Acb,MoraBanc Andorra,Murcia,1.80,Back Home
24791,13:00,Europa Liga Aba,Zadar,Cedevita Olimpija,1.85,Back Home
24802,14:00,França Lnb,Limoges,Cholet,2.40,Back Home
24829,16:00,Eua Ncaa,St. Johns,Quinnipiac,1.64,Back Home
24830,17:30,Eua Ncaa,Minnesota,Nebraska O.,1.67,Back Home
